In [9]:
import configparser
config = configparser.ConfigParser()
config.read('secrets.ini')
custom_vision_key = config['custom_vision']['key']
custom_vision_url = config['custom_vision']['url']
custom_vision_imgurl = config['custom_vision']['imgurl']

In [10]:
import requests
import json
import base64

In [11]:
dataheaders = {'Content-Type': 'application/octet-stream', 'Prediction-Key': custom_vision_key}
data = open('anpr_samples/hsv39.jpg', 'rb').read()

custom_vision_resp = requests.post(url=custom_vision_imgurl, data=data, headers=dataheaders).json()
print(custom_vision_resp)

{'id': '56c40745-6538-4fe3-8829-acad0a789708', 'project': '2c4e0486-88ee-4369-b033-dd24e1e07885', 'iteration': 'e48ad707-5e08-47dd-9273-396ce4a4afa7', 'created': '2020-06-01T08:23:16.733Z', 'predictions': [{'probability': 0.0160742067, 'tagId': '1752966c-d1e1-46f5-924b-505d90a16981', 'tagName': 'plate', 'boundingBox': {'left': 0.004565847, 'top': 0.240038842, 'width': 0.0257141441, 'height': 0.0753260553}}, {'probability': 0.115149885, 'tagId': '1752966c-d1e1-46f5-924b-505d90a16981', 'tagName': 'plate', 'boundingBox': {'left': 0.00208997354, 'top': 0.278702438, 'width': 0.0318060964, 'height': 0.116016328}}, {'probability': 0.0116057871, 'tagId': '1752966c-d1e1-46f5-924b-505d90a16981', 'tagName': 'plate', 'boundingBox': {'left': 0.56674093, 'top': 0.323081523, 'width': 0.07001418, 'height': 0.0435321629}}, {'probability': 0.2426998, 'tagId': '1752966c-d1e1-46f5-924b-505d90a16981', 'tagName': 'plate', 'boundingBox': {'left': 0.8473767, 'top': 0.2650637, 'width': 0.04177034, 'height': 0.

In [53]:
import pandas as pd

hit = pd.DataFrame(custom_vision_resp['predictions']).sort_values(by='probability',ascending=False).head(1).to_dict()
print(hit)

{'probability': {8: 0.9894904}, 'tagId': {8: '1752966c-d1e1-46f5-924b-505d90a16981'}, 'tagName': {8: 'plate'}, 'boundingBox': {8: {'left': 0.630741656, 'top': 0.692196131, 'width': 0.09549934, 'height': 0.07045704}}}


In [19]:
import cv2
import numpy as np
from io import BytesIO

In [37]:
np.array(bytearray(data), dtype='uint8')

array([255, 216, 255, ...,   1, 255, 217], dtype=uint8)

In [38]:
img = cv2.imdecode(np.array(bytearray(data), dtype='uint8'), cv2.IMREAD_COLOR)

In [54]:
item = hit['boundingBox'].popitem()

In [57]:
tagid, boundingbox = item

In [151]:
l,t,w,h = (boundingbox['left'], boundingbox['top'], boundingbox['width'], boundingbox['height'])
print(l,t,w,h)
print('width, height')
print(img.shape[1],img.shape[0])
print([[l,t],[l+w,t],[l,t+h],[l+w,t+h]])
polylines1 = np.multiply([[l,t],[l+w,t],[l+w,t+h],[l,t+h]], [img.shape[1],img.shape[0]])

0.630741656 0.692196131 0.09549934 0.07045704
1280 781
[[0.630741656, 0.692196131], [0.726240996, 0.692196131], [0.630741656, 0.762653171], [0.726240996, 0.762653171]]


In [145]:
polylines = [[800,550],[920,550], [920,600], [800,600]]
img2 = cv2.polylines(img, np.int32([polylines1]), 1, (255,255,0), 4, lineType=cv2.LINE_AA )

In [162]:
img3 = [int(t),int(t+h), int(l),int(l+w)]


In [160]:
cv2.imshow('img3', img3)
cv2.imshow('img2', img2)


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [148]:
cv2.waitKey(0)
cv2.destroyAllWindows()

In [149]:
np.int32([polylines])

array([[[800, 550],
        [920, 550],
        [920, 600],
        [800, 600]]])